In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch import autograd
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.datasets as datasets
from torchvision import transforms
from torch.autograd import Variable

In [2]:
# import datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_data, train_labels = zip(*mnist_trainset)
test_data, test_labels = zip(*mnist_testset)

# # split data into minibatches
# train_loader = DataLoader(dataset=mnist_trainset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(dataset=mnist_testset, batch_size=batch_size, shuffle=True)

In [41]:
# data processing
# flatten 28x28 imgs to 784x1 vectors

# turn into tensors?
flattened_train_data = []

print("Shape before: ", train_data[0].shape)

for i in range(len(train_data)):
    flattened_train_data.append(torch.tensor(torch.reshape(train_data[i], (1, 28*28))))   

print("Shape after: ", flattened_train_data[0].shape)

# tuple --> list --> tensor
train_labels = torch.stack(list(train_labels))
test_labels = torch.stack(list(test_labels))

flattened_train_data = torch.stack(flattened_train_data)
# print(type(train_labels))

Shape before:  torch.Size([1, 28, 28])
Shape after:  torch.Size([1, 784])


In [68]:
# flatten test data

flattened_test_data = []

print("Shape before: ", test_data[0].shape)

for i in range(len(test_data)):
    flattened_test_data.append(torch.tensor(torch.reshape(test_data[i], (1, 28*28))))   

print("Shape after: ", flattened_test_data[0].shape)

flattened_test_data = torch.stack(flattened_test_data)

Shape before:  torch.Size([1, 28, 28])
Shape after:  torch.Size([1, 784])


In [69]:
# given batch number from, return batch of data
def train_batchify(batch_num, batch_size):
    start_index = batch_num * batch_size
    data = flattened_train_data[start_index:start_index + batch_size]
    labels = train_labels[start_index:start_index + batch_size]  
    
    print(start_index, start_index + batch_size)
    
    return (labels, data)

labels, data = train_batchify(100, 100)
print(labels.__len__())
print(data.__len__())

10000 10100
100
100


In [70]:
# # given batch number from, return batch of data
# def test_batchify(batch_num, batch_size):
#     start_index = batch_num * batch_size
#     data = flattened_test_data[start_index:start_index + batch_size]
#     labels = test_labels[start_index:start_index + batch_size]  
    
#     print(start_index, start_index + batch_size)
    
#     return (labels, data)

# labels, data = test_batchify(100, 100)
# print(labels.__len__())
# print(data.__len__())

In [71]:
# hyperparameters
input_size = 784
num_classes = 10

# define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 layer
        self.layer_one = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        x = F.softmax(x)
        return x
    
    

In [72]:
# train model
classifier = Net()
optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.5)

loss_history = []
accuracy_history = []

def train(epoch):
    classifier.train() # training mode
    
    batch_size = 1000
    max_batch_num = len(train_labels) // batch_size
    
    for batch_num in range(0, max_batch_num + 1):
        labels, data = train_batchify(batch_num, batch_size)
        
        labels = Variable(labels)
        data = Variable(data)
        
        optimizer.zero_grad()
        predictions = classifier(data)
        loss = nn.CrossEntropyLoss(predictions, labels)
        loss.backward()
        
        loss_history.append(loss.data[0])
        optimizer.step()
        
        if batch_id == max_batch_num:
            print(loss.data[0])
    

In [73]:
def test(epoch):
    classifier.eval() # test mode
    test_loss = 0
    correct = 0
    
    for i in range(len(test_labels)):
        data = Variable(flattened_test_data[i], volatile=True)
        data = Variable(test_label[i])
        
        output = classifier(data)
        test_loss += nn.CrossEntropyLoss(output, target).data[0]
        prediction = output.data.max(1)[1] # index of max-log probability
        correct += prediction.eq(data).cpu().sum()
        
    test_loss = test_loss
    test_loss /= len(test_labels)
    acc_history.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_labels),accuracy))
    

for epoch in range(0, 3):
    print("epoch: ", epoch)
    train(epoch)
    test(epoch)

epoch:  0
0 1000


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


RuntimeError: bool value of Tensor with more than one value is ambiguous